<a href="https://colab.research.google.com/github/mokoron/EDA/blob/master/en_wordforms_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* not all the words are covered by this inflector, there exists an additional 
function `getInflection(word.text, word.tag_, inflect_oov=True)` that returns a paradigm even for the words that are not presented in the database. we could use it, so that all words will have a paradigm
* current version of disa;biguation is far from perfect, so almost synonyms are added (we check POS and number of intersections of a definition and a sentence)

In [ ]:
%%capture
!pip3 install pyinflect
!python -m spacy download en_core_web_lg
import nltk
nltk.download('punkt')

In [ ]:
import spacy
from pyinflect import getAllInflections
import requests
from bs4 import BeautifulSoup
import en_core_web_lg
from nltk.tokenize import word_tokenize
nlp = en_core_web_lg.load()
add_inflections = {
    'i' : ['i', 'me'],
    'me' : ['i', 'me'],
    'we' : ['we', 'us'],
    'us' : ['we', 'us'],
    'my' : ['my', 'mine'],
    'mine' : ['my', 'mine'],
    'our' : ['our', 'ours'],
    'ours' : ['our', 'ours'],
    'your' : ['your', 'yours'],
    'yours' : ['your', 'yours'],
    'he' : ['he', 'him'],
    'him' : ['he', 'him'],
    'she' : ['she', 'her'],
    'her' : ['she', 'her'],
    'her' : ['her', 'hers'],
    'hers' : ['her', 'hers'],
    'they' : ['they', 'them'],
    'them' : ['they', 'them'],
    'their' : ['their', 'theirs'],
    'theirs' : ['their', 'theirs'],
}
def get_synonyms(lemma, postag, sentence):
  trans = str.maketrans({x:'' for x in list('1234567890.')})
  url = f'https://www.synonym.com/synonyms/{lemma}'
  data = BeautifulSoup(requests.get(url).text, 'html.parser')
  pos_dct = {'adjective': 'ADJ',
             'noun':'NOUN',
             'verb': 'VERB',
             'adverb':'ADV'}
  if data.find('div', {'class': "not-found-message"}):
    return [lemma]
  else:
    pos = [pos_dct[span.text.translate(trans).strip()] for span in data.findAll('span', {'class': "part-of-speech"})]
    syns = [[sec.text.strip() for sec in section.findAll('a')] for section in data.findAll('div', {'class': 'type-synonym'})]
    antons = [[sec.text.strip() for sec in section.findAll('a')] for section in data.findAll('div', {'class': 'type-antonym'})]
    words = [x.text.translate(trans).strip() for x in data.findAll('h2', {'class': "mdc-typography mdc-typography--headline2"})]
    defs = [word_tokenize(x.text) for x in data.findAll('span', {'class': "definition"})]
    synsans = [[z, u] for x,y,z,w,u in zip(words, pos, syns, defs, antons) if x == lemma and y == postag and len(set(w) & set(sentence)) != 0]
    return list(set(sum([x[0] for x in synsans], []))), list(set(sum([x[1] for x in synsans], [])))
def get_words_paradigms_ner(sentences):
  sentence = nlp(sentences)
  paradigms = []
  ners = []
  synonyms = []
  antonyms = []
  for word in sentence:
    if word.tag_[0] in ['V', 'A', 'N']:
      members = list(set([x[0] for x in getAllInflections(word.lemma_, pos_type=word.tag_[0]).values()]))
      if members:
        paradigms.append([word.text, members])
      else:
        paradigms.append([word.text, [word.text]])
    elif word.text in add_inflections:
      paradigms.append([word.text, add_inflections[word.text]])
    else:
      paradigms.append([word.text, [word.text]])
    ners.append([word.text, word.ent_type_])
    if word.pos_ in ['ADJ', 'ADV', 'NOUN', 'VERB']:
      syns, ans = get_synonyms(word.lemma_, word.pos_, word_tokenize(sentences))
      synonyms.append([word.text, syns])
      antonyms.append([word.text, ans])
    else:
      synonyms.append([word.text, [word.text]])
      antonyms.append([word.text, []])
  return paradigms, ners, synonyms, antonyms

In [ ]:
par, ner, syn, ans = get_words_paradigms_ner('SpaceX is an aerospace manufacturer and space transport services company headquartered in California. It was founded in 2002 by entrepreneur and investor Elon Musk with the goal of reducing space transportation costs and enabling the colonization of Mars.')

In [ ]:
import plotly.figure_factory as ff
import pandas as pd
colorscale = [[0, '#53C41E'],[.5, '#C0FFA3'],[1, '#ffffff']]
df = pd.DataFrame({'word': [x[0] for x in par], 'paradigm':[x[1] for x in par], 'NER':[x[1] for x in ner], 'synonyms':[x[1] for x in syn], 'antonyms':[x[1] for x in ans]})
fig = ff.create_table(df, colorscale=colorscale)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 8
fig.show()

In [ ]:
df.head(df.shape[0])

,word,paradigm,NER,synonyms,antonyms
0,SpaceX,[SpaceX],ORG,[SpaceX],[]
1,is,"[am, being, was, be, is, been]",,[is],[]
2,an,[an],,[an],[]
3,aerospace,"[aerospaces, aerospace]",,"[region, infinite, part, space, outer space]","[inside, end, beginning, outside, misconception]"
4,manufacturer,"[manufacturer, manufacturers]",,"[concern, automaker, car manufacturer, busines...","[unconcern, inactivity]"
5,and,[and],,[and],[]
6,space,"[space, spaces]",,"[air space, infinite, interval, area, hair spa...","[inactivity, uncover, studio, deglycerolize, s..."
7,transport,"[transport, transports]",,"[tram, freight, passive transport, tape deck, ...","[electronic mail, email, fall, natural object,..."
8,services,"[service, services]",,"[public service, watch night, naval forces, na...","[disallow, merit, irresponsibleness, worthless..."
9,company,"[companies, company]",,"[packaging company, food company, shipping com...","[misfortune, nonmember, inactivity, keep down,..."
